## Basic settings

### Load libraries

In [ ]:
# Core libraries
import numpy as np
from matplotlib import pyplot as plt
#plt.style.use('default') # Use for black theme with Visual Studio
import matplotlib.patches as mpatches
import matplotlib.path as mpath
import pandas as pd

# Image IO
#! pip install ncempy #Install ncempy for loading *.ser file
import ncempy.io as nio

# Image filters
from skimage.filters import gaussian
from skimage.morphology import opening, closing, dilation, square
from skimage.measure import label, regionprops

# Processing
from tqdm import tqdm #Tracking the iterations
import time

In [ ]:
# Graph parameters for video and publication

import matplotlib.pylab as pylab
params = {
        'legend.fontsize': 'x-large',
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large',
        'axes.edgecolor':'black', 
        'xtick.color':'black',
        'ytick.color':'black', 
        'figure.facecolor':'white'
        }
pylab.rcParams.update(params)

### Functions

In [ ]:
# Draw scalebar in TEM image

def scalebar(ax, scale=(1*10**(-6)), img_w=1024, img_h=1024, length=10, unit='µm', color='white', fontsize=15, text=True): 
    # input scale in [m] unit 
    # display unit as inserted parameter
    if unit=='µm':
        scale=scale*10**6
    elif unit=='nm':
        scale=scale*10**9
    else:
        scale=scale
        
    scale_text=str(length)+' '+unit
    scalebar=mpatches.Rectangle((img_w*0.1, img_h*0.9), (1/scale*length), (img_h*0.02), edgecolor='None', facecolor=color, fill='True')
    ax.add_patch(scalebar)
    if text:
        ax.text((img_w*0.1+0.5*(1/scale*length)), (img_h*0.9-img_h*0.05), scale_text, horizontalalignment='center', fontsize=fontsize, color=color)

# Part 1. Cyclic Voltametry

## Load data

In [ ]:
img_file='data\Zn-10_1.ser'

imgs=nio.read(img_file) # load file
img_num, img_w, img_h = imgs['data'].shape
print('Number of images: ' + str(img_num))
print('Size of image: ' + str(img_w) + ' x ' + str(img_h))

scale=imgs['pixelSize'][0]
print('1 pixel: '+str(scale) + ' ' +imgs['pixelUnit'][0])

fig=plt.figure()
ax=fig.add_subplot(111)
ax.imshow(imgs['data'][100], cmap='gray')
ax.axis('off')
scalebar(ax, scale=scale, img_w=img_w, img_h=img_h, length=2)

## Figure 1

### STEM image series

In [ ]:
img_list=[52, 92, 101, 111, 134, 150, 177]
vol_list=['0.0V', '-1.2V', '-1.5V', '-1.2V', '-0.5V', '0.0V', '0.8V']

fig = plt.figure(figsize=(8,4), dpi=300)
for i in range(len(img_list)):
    ax = fig.add_subplot(2, 4, i+1)
    ax.imshow(imgs['data'][img_list[i]], cmap='gray')
    #ax.text(50, 100, vol_list[i], color='white', fontsize=10)
    ax.axis('off')
    scalebar(ax, scale=scale, img_w=img_w, img_h=img_h, length=2, text=False)

### CV graph

In [ ]:
csv_file='data/Zn_10_area_single_cycle.csv'
csv_df=pd.read_csv(csv_file)

fig = plt.figure(figsize=(5, 5), dpi=300)

labelsize=25
linewidth=2

ax = fig.add_subplot(111)
#axs[0].tick_params(labelsize=labelsize)
ax.plot(csv_df['Voltage'], csv_df['Current'], color='black', linewidth=linewidth)
ax.set_xlim(-2, 1)
ax.set_xticks(np.arange(-2, 1.1, 1))
plt.xticks(fontsize=labelsize)
ax.set_xlabel('Potential vs Pt [V]', fontsize=labelsize)
ax.set_ylim(-5, 5)
ax.set_yticks(np.arange(-5, 5.1, 1))
plt.yticks(fontsize=labelsize)
ax.set_ylabel('Current [µA]', fontsize=labelsize)
spot_list=[0, 39, 48, 58, 81, 97, 123]
for i in spot_list:
    ax.scatter(csv_df['Voltage'][i], csv_df['Current'][i], s=100, facecolor='black')

## Figure 2

### Thresholding

In [ ]:
img_list=[92, 101, 111, 134, 150, 177]

fig = plt.figure(figsize=(6,5), dpi=300, constrained_layout=True)
for i in range(len(img_list)):
  ax = fig.add_subplot(2, 3, i+1)
  gaus_subt_img=(gaussian(imgs['data'][img_list[i]], sigma=3)-gaussian(imgs['data'][0], sigma=3))
  ax.imshow(gaus_subt_img>0.1, cmap='gray')
  ax.axis('off')  

#### Thresholding issue

In [ ]:
## Subtraction way

img_list=[92, 101, 111, 134, 150, 177]
vol_list=['-1.2V', '-1.5V', '-1.2V', '-0.5V', '0.0V', '0.8V']

fig, axs = plt.subplots(6, 6, figsize=(10,10), dpi=300, constrained_layout=True)
for i in range(len(img_list)):
    axs[i][0].imshow(imgs['data'][img_list[i]], cmap='gray')
    axs[i][0].axis('off')
    scalebar(axs[i][0], scale=scale, img_w=img_w, img_h=img_h, length=2, text=False)
    
    gaus_subt_img=(gaussian(imgs['data'][img_list[i]], sigma=3)-gaussian(imgs['data'][0], sigma=3))
    axs[i][1].imshow(gaus_subt_img, cmap='gray')
    axs[i][1].axis('off')
    
    thres=0.1
    axs[i][2].hist(gaus_subt_img.ravel(), bins=100, range=[-0.1, 0.5])
    axs[i][2].set_ylim(0,200000)
    axs[i][2].plot([thres-0.05, thres-0.05],[0,200000], linestyle='dashed', linewidth=1, color='black')
    axs[i][2].plot([thres, thres],[0,200000], linestyle='solid', linewidth=1, color='black')
    axs[i][2].plot([thres+0.05, thres+0.05],[0,200000], linestyle='dotted', linewidth=1, color='black')
    
    axs[i][3].imshow(gaus_subt_img>thres-0.05, cmap='gray')
    axs[i][3].axis('off')
    
    axs[i][4].imshow(gaus_subt_img>thres, cmap='gray')
    axs[i][4].axis('off')
    
    axs[i][5].imshow(gaus_subt_img>thres+0.05, cmap='gray')
    axs[i][5].axis('off')

In [ ]:
## Division way

img_list=[92, 101, 111, 134, 150, 177]
vol_list=['-1.2V', '-1.5V', '-1.2V', '-0.5V', '0.0V', '0.8V']

fig, axs = plt.subplots(6, 6, figsize=(10,10), dpi=300, constrained_layout=True)
for i in range(len(img_list)):
    axs[i][0].imshow(imgs['data'][img_list[i]], cmap='gray')
    axs[i][0].axis('off')
    scalebar(axs[i][0], scale=scale, img_w=img_w, img_h=img_h, length=2, text=False)
    
    gaus_div_img=(gaussian(imgs['data'][img_list[i]], sigma=3)/gaussian(imgs['data'][0], sigma=3))
    axs[i][1].imshow(gaus_div_img, cmap='gray')
    axs[i][1].axis('off')
    
    thres=2
    axs[i][2].hist(gaus_div_img.ravel(), bins=100, range=[0, 10])
    axs[i][2].set_ylim(0,300000)
    axs[i][2].plot([thres*0.75, thres*0.75],[0,300000], linestyle='dashed', linewidth=1, color='black')
    axs[i][2].plot([thres, thres],[0,300000], linestyle='solid', linewidth=1, color='black')
    axs[i][2].plot([thres*1.25, thres*1.25],[0,300000], linestyle='dotted', linewidth=1, color='black')
    
    axs[i][3].imshow(gaus_div_img>(thres*0.75), cmap='gray')
    axs[i][3].axis('off')
    
    axs[i][4].imshow(gaus_div_img>thres, cmap='gray')
    axs[i][4].axis('off')
    
    axs[i][5].imshow(gaus_div_img>(thres*1.25), cmap='gray')
    axs[i][5].axis('off')

### Area vs Time graph

In [ ]:
#############################
# Area vs Frame graph update
#############################

csv_file='data/Zn_10_area_single_cycle.csv'
csv_df=pd.read_csv(csv_file)
csv_df['Area']=0

for i in tqdm(range(150)):
  cur_num=52+i
  pro_img=gaussian(imgs['data'][cur_num], sigma=3)-gaussian(imgs['data'][0], sigma=3)

  dep_area=np.sum(pro_img>0.1)*(scale*10**6)**2 #[um2]
  csv_df.iloc[i,2]=dep_area

csv_df.head()

In [ ]:
#############################
# Area vs Frame graph update
#############################

fig = plt.figure(figsize=(4, 7))
fig.set_dpi(300)
labelsize=15
linewidth=2

axs = fig.subplots(3,1, sharex=True)
axs[0].tick_params(labelsize=labelsize)
axs[0].plot(np.arange(0,45.3,0.3), csv_df['Area'], color='black', linewidth=linewidth)
axs[0].set_ylim(0, 30)
axs[0].set_ylabel('Deposition [µm^2]', fontsize=labelsize)

axs[1].plot(np.arange(0,45.3,0.3), csv_df['Current'], color='black', linewidth=linewidth)
axs[1].tick_params(labelsize=labelsize)
axs[1].set_ylim(-5, 5)
axs[1].set_ylabel('Current [nA]', fontsize=labelsize)

axs[2].plot(np.arange(0,45.3,0.3), csv_df['Voltage'], color='black', linewidth=linewidth)
axs[2].tick_params(labelsize=labelsize)
axs[2].set_ylim(-2, 1)
axs[2].set_ylabel('Potential [V]', fontsize=labelsize)

axs[2].set_xlim(0, 50)
axs[2].set_xticks(np.arange(0, 55, 10))
axs[2].set_xlabel('Time [s]', fontsize=labelsize)


## SI

### Figure S2. Explanation of image processing

In [ ]:
# Full cycle images : 52 ~ 202

cur_img=120

ori_img=imgs['data'][cur_img]
gaus_img=gaussian(ori_img, sigma=3)
ref_img=gaussian(imgs['data'][0], sigma=3)
pro_img=gaus_img-ref_img

fig = plt.figure(figsize=plt.figaspect(0.6), layout='tight')

ax1=fig.add_subplot(2,3,1)
ax1.imshow(ori_img, cmap='gray')
ax1.set_title('(a) Original')
ax1.axis('off')

ax2=fig.add_subplot(2,3,2)
ax2.imshow(imgs['data'][0], cmap='gray')
ax2.set_title('(b) Reference')
ax2.axis('off')
scalebar(ax2, scale=scale, img_w=img_w, img_h=img_h, length=2)

ax3=fig.add_subplot(2,3,3)
ax3.imshow(pro_img, cmap='gray')
ax3.set_title('(c) Gaussian & Subtraction')
ax3.axis('off')

ax4=fig.add_subplot(2,3,4)
ax4.hist(pro_img.ravel(), bins=255, range=(0.001,0.5))
ax4.set_xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])
ax4.set_ylim(0, 30000)
ax4.set_aspect(0.00002)
ax4.set_xticklabels(ax4.get_xticks(), rotation=45)
ax4.set_title('(d) Histogram')
ax4.plot([0.1, 0.1],[0, 30000], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]

ax5=fig.add_subplot(2,3,5)
ax5.imshow(pro_img>0.1, cmap='gray')
ax5.set_title('(e) Threshold (>0.1)')
ax5.axis('off')

ax6=fig.add_subplot(2,3,6, projection='3d')
ax6.set_title('(f) 3d visualization')

x = np.arange(1024)
y = np.arange(1024)
x, y = np.meshgrid(x, y)

color=ax6.plot_surface(x, y, pro_img, cmap='jet', vmin=0, vmax=0.5)
ax6.set_zlim(0,2)
ax6.invert_yaxis()
ax6.view_init(60, 90)#(70,-90) #(60,90)
ax6.axis('off')
ax6.dist=6
#cb_ax=ax6.add_axes([0, 0.1, 0.2, 0.3, 0.4, 0.5])
cbar=fig.colorbar(color, ax=ax6, aspect=10)
cbar.ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])

plt.subplots_adjust(hspace=0.5, wspace=0.1)

plt.show()

### Figure S3. Potential for volume quantification via 3d projection

In [ ]:
x = np.arange(1024)
y = np.arange(1024)
x, y = np.meshgrid(x, y)

fig = plt.figure()
fig.set_dpi(300)
img_list=[92, 101, 111, 134, 150, 177]
for i in range(len(img_list)):
  ax = fig.add_subplot(2, 3, i+1, projection='3d')
  gaus_subt_img=(gaussian(imgs['data'][img_list[i]], sigma=3)-gaussian(imgs['data'][0], sigma=3))
  color=ax.plot_surface(x, y, gaus_subt_img, cmap='jet', vmin=0, vmax=0.5)
  ax.set_zlim(0,1)
  ax.invert_yaxis()
  ax.view_init(60,90)
  ax.axis('off')
  ax.dist=6

cax = plt.axes([0.93, 0.2, 0.03, 0.6])
fig.colorbar(color, cax=cax, ticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5])#.set_label(label='label', size=20)
color.figure.axes[1].tick_params(labelsize=10)

## Movie S1.

In [ ]:
import matplotlib.pylab as pylab
import matplotlib.patches as mpatches
plt.style.use('default')
params = {
        'legend.fontsize': 'x-large',
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large',
        'axes.edgecolor':'black', 
        'xtick.color':'black',
        'ytick.color':'black', 
        'figure.facecolor':'white'
        }
pylab.rcParams.update(params)

from matplotlib import animation as ani
fig = plt.figure(figsize=(10, 10))
plt.subplots_adjust(wspace=0.3)


ax1 = fig.add_subplot(221)
ax1.axis('off')
img_plot_ori=ax1.imshow((imgs['data'][1]), cmap='gray')

img_w, img_h = imgs['data'][1].shape
scalebar(ax1, scale=scale, img_w=img_w, img_h=img_h, length=2)

ax2 = fig.add_subplot(222)
ax2.set_ylabel('Current [nA]')
ax2.set_xlabel('Voltage [V]')
ax2.set_xlim(-2, 1)
ax2.set_ylim(-5, 5)
plot_cv, =ax2.plot([], [], lw=2, c='black')

ax3 = fig.add_subplot(223)
ax3.axis('off')
img_plot_pro=ax3.imshow((imgs['data'][1]), cmap='gray')

ax4 = fig.add_subplot(224) # (rows, columns, nth figure)
ax4.set_ylabel('Deposition [µm^2]')
ax4.set_xlabel('Time [s]')
ax4.set_xlim(0, 50) #minute, full 120
ax4.set_ylim(0, 30)
plot_area, =ax4.plot([], [], lw=2, c='black')

length=150

time=np.arange(0,45.3,0.3)

import tqdm.notebook as tqdm
pbar= tqdm.tqdm(total=length) #progress bar

def animate(i):   
    ori_img=(imgs['data'][52+i])
    gaus_img=gaussian(ori_img, sigma=3)
    ref_img=gaussian((imgs['data'][0]), sigma=3)
    pro_img=(gaus_img-ref_img)>0.1
    
    img_plot_ori.set_data(ori_img)
    img_plot_ori.autoscale()

    img_plot_pro.set_data(pro_img)
    img_plot_pro.autoscale()

    plot_cv.set_data(csv_df['Voltage'][0:i+1], csv_df['Current'][0:i+1])

    plot_area.set_data(time[0:i+1], csv_df['Area'][0:i+1])

    pbar.update(1)
    
    
movie=ani.FuncAnimation(fig, animate, length, interval=50, repeat=False) #blit

movie.save('results\Zn deposition.avi')


# Part 2. Dendritic growth

## Load data

In [ ]:
img_file='data\Zn-14_1.ser'

imgs=nio.read(img_file) # load file
img_num, img_w, img_h = imgs['data'].shape
print('Number of images: ' + str(img_num))
print('Size of image: ' + str(img_w) + ' x ' + str(img_h))

scale=imgs['pixelSize'][0]
print('1 pixel: '+str(scale) + ' ' +imgs['pixelUnit'][0])

fig=plt.figure()
ax=fig.add_subplot(111)
ax.imshow(imgs['data'][100], cmap='gray')
ax.axis('off')
scalebar(ax, scale=scale, img_w=img_w, img_h=img_h, length=2)

## Figure 3

### Original STEM images

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3), dpi=300, constrained_layout=True, squeeze=False)

frame_shift=2
fps=3.3 
step=1 #[s]
sigma=3
frame_num=4
time_shift=6 #[s]

for i in range(3):

    num1=int((i+time_shift+1)*step*fps+frame_shift)

    axs[0,i].imshow(imgs['data'][num1], cmap='gray')
    axs[0,i].axis('off')
    scalebar(axs[0,i], scale=scale, img_w=img_w, img_h=img_h, length=2, fontsize=15)


### Graph (Volume vs Time) with calibration

In [ ]:
df_static = pd.DataFrame(columns=['Frame', 'second', 'area', 'volume'])

threshold=0.1

frame_shift=2
fps=3.3 
step=1 #[s]
sigma=3

for i in tqdm(range(10)):

    num1=int((i+1)*step*fps+frame_shift)

    diff=gaussian(imgs['data'][num1], sigma=sigma) - gaussian(imgs['data'][0], sigma=sigma)
    
    ### surface
    
    area=np.sum(diff>threshold) * (scale*1000000) * (scale*1000000) ## unit=[um^2] ## scale = xx m/pixel
    #print(cur_surface_area)
    
    volume=np.sum(diff*(diff>threshold)) * (scale*1000000) * (scale*1000000) * 1 ##unit=[um^3] ## scale = xx m/pixel
    #print(cur_surface_vol)

    df_static.loc[len(df_static)]=[num1, (i+1), area, volume]

#plt.plot(df_static['second'], df_static['volume'])

fig=plt.figure(figsize=(3,3), dpi=300)
ax1=fig.add_subplot(111)

ax1.plot(df_static['second'], df_static['volume'], color='black', linewidth=2)

ax1.set_xlim(0, 10)
ax1.set_xticks(np.arange(0, 11, 2))
ax1.set_ylim(0, 3)
ax1.set_yticks(np.arange(0, 3.5, 0.5))

ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Deposition [µm^3]')

ax1.plot([7, 7],[0, 3], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]
ax1.plot([8, 8],[0, 3], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]
ax1.plot([9, 9],[0, 3], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]


### Dynamic analysis

In [ ]:
## at high current (-5 uA) 10 s
frame_shift=2
fps=3.3 
step=1 #[s]
sigma=3
frame_num=3
time_shift=7 #[s]

#fig=plt.figure(figsize=(10,10), constrained_layout=True)

fig, axs = plt.subplots(1, frame_num, figsize=(frame_num*3, 3), dpi=300,
                        constrained_layout=True, squeeze=False)

for i in range(frame_num):
    col=int(i%frame_num)
    row=int(i/frame_num)
    num1=int((i+time_shift+1)*step*fps+frame_shift)
    num2=int((i+time_shift)*step*fps+frame_shift)
    print((i+time_shift+1)*step)
    print(num1)
    diff=gaussian(imgs['data'][num1], sigma=sigma) - gaussian(imgs['data'][num2], sigma=sigma)
    vmax=0.5
    vmin=0
    img_locate=axs[row, col].imshow(diff, cmap='jet', vmax=vmax, vmin=vmin)
    axs[row, col].axis('off')

plt.colorbar(img_locate, orientation='vertical')
#plt.savefig(r'C:\Users\j.park\Documents\python\Zn deposition\results\Plating at both current.png')

In [ ]:
df_static = pd.DataFrame(columns=['Frame', 'second', 'area', 'volume'])

threshold=0.1

frame_shift=2
fps=3.3 
step=1 #[s]
sigma=3

for i in tqdm(range(10)):

    num1=int((i+1)*step*fps+frame_shift)
    num2=int((i)*step*fps+frame_shift)
    diff=gaussian(imgs['data'][num1], sigma=sigma) - gaussian(imgs['data'][num2], sigma=sigma)
    
    ### surface
    
    area=np.sum(diff>threshold) * (scale*1000000) * (scale*1000000) ## unit=[um^2] ## scale = xx m/pixel
    #print(cur_surface_area)
    
    volume=np.sum(diff*(diff>threshold)) * (scale*1000000) * (scale*1000000) * 1 ##unit=[um^3] ## scale = xx m/pixel
    #print(cur_surface_vol)

    df_static.loc[len(df_static)]=[num1, (i+1), area, volume]

#plt.plot(df_static['second'], df_static['volume'])

fig=plt.figure(figsize=(3,3), dpi=300)
ax1=fig.add_subplot(111)

ax1.plot(df_static['second'], df_static['volume'], color='black', linewidth=2)

ax1.set_xlim(0, 10)
ax1.set_xticks(np.arange(0, 11, 2))
ax1.set_ylim(0,1)

ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Deposition per 1 s [µm^3]')

ax1.plot([7, 7],[0, 1], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]
ax1.plot([8, 8],[0, 1], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]
ax1.plot([9, 9],[0, 1], color='black', linewidth=1, linestyle='dashed') # [x1, x2], [y1, y2]

#plt.savefig(r'C:\Users\j.park\Documents\python\Zn deposition\results\Deposition graph.png', dpi=fig.dpi)


### Tracking the growth via profiles

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 3), dpi=300, constrained_layout=True, squeeze=False)
vmin=0
vmax=0.5
axs[0,0].set_ylim(vmin,vmax)
axs[0,1].set_xlim(vmin,vmax)
axs[0,2].set_xlim(vmin,vmax)

axs[0,0].set_aspect(10/0.5)
axs[0,1].set_aspect(0.5/10)
axs[0,2].set_aspect(0.5/10)

axs[0,0].set_xlabel('X: Across dendrite [µm]')
axs[0,0].set_ylabel('Z: Thickness [µm]')
axs[0,1].set_xlabel('Z: Thickness [µm]')
axs[0,1].set_ylabel('Y: Perpendicular to dendrite [µm]')
axs[0,2].set_xlabel('Z: Thickness [µm]')
axs[0,2].set_ylabel('Y: Close to electrode [µm]')

sigma=3
frame_shift=2
fps=3.3 
step=1 #[s]
frame_num=10

pos_top=0
pos_bottom=1024
pos_line=[450,100,660] #yz_near, yz_far, xz

pos_x_um=((scale*10**6)*1024) # 9.460 um, 1024 pixels

y_axis = np.linspace(10-pos_x_um, 10, 1024) #np.linspace(0, int((scale*10**6)*1024), 1024)

import matplotlib.pylab as pl
colors = pl.cm.jet(np.linspace(0,1, frame_num))

axs[0,0].set_xlim(0, 10) # ~ 10 um
axs[0,1].set_ylim(0, 10)
axs[0,2].set_ylim(0, 10)

from skimage.morphology import dilation, square

for i in range(frame_num):
    diff=gaussian(imgs['data'][int((i+1)*fps*step+frame_shift)], sigma=sigma) - gaussian(imgs['data'][0], sigma=sigma)
    axs[0,0].plot(y_axis, diff[pos_line[2],pos_top:pos_bottom], color=colors[i]) 
    axs[0,1].plot(diff[pos_top:pos_bottom,pos_line[1]], y_axis, color=colors[i])
    axs[0,2].plot(diff[pos_top:pos_bottom,pos_line[0]], y_axis, color=colors[i])   
    
axs[0,1].invert_xaxis()
axs[0,1].invert_yaxis()
axs[0,2].invert_xaxis()
axs[0,2].invert_yaxis()

pos_y_labels = [10, 8, 6, 4, 2, 0]
axs[0,1].set_yticklabels(pos_y_labels)
axs[0,2].set_yticklabels(pos_y_labels)

pos_z_tick=[(0.5-j*0.1) for j in range(6)] # equation + for loop
axs[0,1].set_xticks(pos_z_tick)
axs[0,2].set_xticks(pos_z_tick)
pos_z_labels = [0.5, 0.4, 0.3, 0.2, 0.1, 0]
axs[0,1].set_xticklabels(pos_z_labels)
axs[0,2].set_xticklabels(pos_z_labels)

#plt.savefig(r'C:\Users\j.park\Documents\python\Zn deposition\results\Plating series.png')

In [ ]:
from skimage.feature import canny

projection_img=np.zeros((img_h, img_w), dtype=float)

for i in range(frame_num):
    diff=gaussian(imgs['data'][int((i+1)*fps*step+frame_shift)], sigma=sigma) - gaussian(imgs['data'][0], sigma=sigma)   
    edge_img=dilation(canny(diff>0.15, sigma=3).astype(int), square(5))
    projection_img=np.ma.array(projection_img, mask=edge_img, fill_value=((i+1))).filled()

fig, axs = plt.subplots(1, 1, figsize=(3, 3),dpi=300, constrained_layout=True, squeeze=False)
projection_img = np.ma.array(projection_img, mask=projection_img<1, fill_value=np.nan)#.filled()#.astype(int)
axs[0,0].imshow(imgs['data'][0], cmap='gray', alpha=0.3)
axs[0,0].imshow(projection_img, interpolation = 'none', cmap='jet', vmin=0, vmax=frame_num) 

axs[0,0].plot([pos_line[0], pos_line[0]],[pos_top, pos_bottom], color='black', linewidth=1.5, linestyle='dashed') # [x1, x2], [y1, y2]
axs[0,0].plot([pos_line[1], pos_line[1]],[pos_top, pos_bottom], color='black', linewidth=1.5, linestyle='dotted')
axs[0,0].plot([pos_top, pos_bottom], [pos_line[2], pos_line[2]], color='black', linewidth=1.5, linestyle='dashdot') # [x1, x2], [y1, y2]
axs[0,0].set_ylim(1024,0)
axs[0,0].set_xlim(0,1024)
axs[0,0].axis('off')

img_w, img_h = imgs['data'][0].shape
scalebar(axs[0,0], scale=scale, img_w=img_w, img_h=img_h, length=2, fontsize=15)

## SI

### Figure S5. Z-contrast Calibration

In [ ]:
cur_num=28 #40

pos_line=244
pos_top=603
cal_length=1
pos_um=int(1/(scale*10**6)*cal_length) # 2 um, 216 pixels
pos_bottom=pos_top+pos_um
print(pos_bottom)

sigma=3
vmin=0
vmax=0.5

diff=gaussian(imgs['data'][cur_num], sigma=sigma) - gaussian(imgs['data'][0], sigma=sigma)

fig=plt.figure(figsize=(10,2), dpi=300)
ax1=fig.add_subplot(121)
show_img=ax1.imshow(diff, cmap='jet', vmin=vmin, vmax=vmax)
ax1.plot([pos_line, pos_line], [pos_top, pos_bottom], color='red', linewidth=2) # [x1, x2], [y1, y2]
ax1.axis('off')
ax1.set_aspect(1)
fig.colorbar(show_img, ax=ax1, ticks=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5])

x_axis = np.linspace(0, cal_length, pos_um)

ax2=fig.add_subplot(122)
ax2.plot(x_axis, diff[pos_top:pos_bottom, pos_line], color='red')
ax2.set_ylim(vmin, vmax)
ax2.set_xlabel('Distance [µm]')
ax2.set_ylabel('Intensity [a.u.]')

ax2.text(0.6, 0.45, 'hemisphere, radius=0.2 µm', horizontalalignment='center', fontsize=10, color='black')

#((x - x0) / a) ** 2 + ((y - y0) / b) ** 2 == 1
radius=0.2
a =  radius   #0.6   #216/4
b =  radius   #0.6   #0.25
x0 = 0.5  #216/2
y0 = 0  #0
x = np.linspace(-a + x0, a + x0)
y = b * np.sqrt(1 - ((x - x0) / a) ** 2) + y0
ax2.plot(x, y, linestyle='--', color='black')

ax2.set_xticks(np.linspace(0, 1, 6))
ax2.set_yticks(np.linspace(0, 0.5, 6))

### Assumption: hemisphere => XY axis 1 um width (diameter) = Z axis 0.5 um height / 1 pixel


## Movie S2

In [ ]:
import matplotlib.pylab as pylab
import matplotlib.patches as mpatches
plt.style.use('default')
params = {
        'legend.fontsize': 'x-large',
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large',
        'axes.edgecolor':'black', 
        'xtick.color':'black',
        'ytick.color':'black', 
        'figure.facecolor':'white'
        }
pylab.rcParams.update(params)

frame_shift=2
fps=3.3 
step=1 #[s]
sigma=3

def diff_img(i):
    
    num1=int((i+1)*step*fps+frame_shift)
    num2=int((i)*step*fps+frame_shift)

    diff=gaussian(imgs['data'][num1], sigma=sigma) - gaussian(imgs['data'][num2], sigma=sigma)

    return diff

from matplotlib import animation as ani
fig = plt.figure(figsize=(10, 6))#, constrained_layout=True)
plt.subplots_adjust(wspace=0.3)

i=1
num1=int((i+1)*step*fps+frame_shift)

vmax=0.5
vmin=0

ax1 = fig.add_subplot(121)
ax1.axis('off')
img_plot_ori=ax1.imshow(imgs['data'][num1], cmap='gray')
ax1.set_title('Orignal')

img_w, img_h = imgs['data'][1].shape
scalebar(ax1, scale=scale, img_w=img_w, img_h=img_h, length=2, fontsize=15)
time=ax1.text((img_w*0.2), (img_h*0.1), str(int((num1-frame_shift)/fps))+ ' s', horizontalalignment='center', fontsize=15, color='white')

ax2 = fig.add_subplot(122) # (rows, columns, 1st figure)
ax2.axis('off')
img_plot_pro=ax2.imshow(diff_img(i), cmap='jet', vmin=vmin, vmax=vmax)
ax2.set_title('Deposition per 1 s')
scalebar(ax2, scale=scale, img_w=img_w, img_h=img_h, length=2, fontsize=15)

e_edge=0
shift_x=-10
shift_y=30
e_shape=[70+e_edge+shift_y, 890-e_edge+shift_y, 470+e_edge+shift_x, 1024-e_edge+shift_x] # electrode shape

img_on_electrode=mpatches.Rectangle((e_shape[2], e_shape[0]), (e_shape[3]-e_shape[2]), (e_shape[1]-e_shape[0]), edgecolor='white', facecolor='None', fill='False', linestyle='dashed')
ax2.add_patch(img_on_electrode)

cax = fig.add_axes([0.91, 0.2, 0.02, 0.6]) #left, bottom, width, height
plt.colorbar(img_plot_pro, cax=cax) #, orientation='vertical')

length=32

import tqdm.notebook as tqdm
pbar= tqdm.tqdm(total=length) #progress bar

def animate(i):   

    frame_shift=2

    num1=i+3 +frame_shift # 3=fps (difference between 1 s)
    num2=i + frame_shift
    second=int((num1-frame_shift)/fps)

    ori_img=imgs['data'][num1]
    img_plot_ori.set_data(ori_img)
    img_plot_ori.autoscale()

    time.set_text(str(second)+ ' s')

    gaus_img=gaussian(ori_img, sigma=sigma)
    ref_img=gaussian(imgs['data'][num2], sigma=sigma)
    pro_img=gaus_img-ref_img
    
    img_plot_pro.set_data(pro_img)
    
    pbar.update(1)
    
movie=ani.FuncAnimation(fig, animate, length, interval=100, repeat=False) #blit

movie.save('results\Dendrite growth.avi')


# Part 3. 4D STEM

## Libertem libraries and functions

In [ ]:
import libertem.api as lt
from libertem.udf import UDF

#libertem server run
ctx = lt.Context()

In [ ]:
#Functions
class SumOfPixels(UDF): #Normally for making virtual HADDF image
    def get_result_buffers(self):
        return{
            'sum_of_pixels': self.buffer(kind='nav', dtype='float32')
        }

    def process_frame(self, frame):
        self.results.sum_of_pixels[:]=np.sum(frame)

class MaxFrameUDF(UDF): #Normally for making diffraction pattern
    def get_result_buffers(self):
        return {
            'maxframe': self.buffer(kind='sig', dtype='float32')
        }

    def process_frame(self, frame):
        # element-wise maximum:
        self.results.maxframe[:] = np.maximum(self.results.maxframe, frame)

    def merge(self, dest, src):
        # src: the maximum observed in the current partition
        # dest: the maximum observed in all partitions that were already merged together
        dest.maxframe[:]=np.maximum(dest.maxframe, src.maxframe)

# Virtual STEM image based on ring analysis with standard deviation
#https://libertem.github.io/LiberTEM/_modules/libertem/udf/FEM.html#run_fem
from libertem.masks import _make_circular_mask
class FEMUDF(UDF): #Fluctuation EM
    
    def __init__(self, center, ri, ro):
        super().__init__(center=center, ri=ri, ro=ro)
    
    def get_result_buffers(self):
        return {
            'intensity': self.buffer(kind='nav', dtype='float32')
        }
    
    def get_task_data(self):
        center = self.params.center
        sig_shape = tuple(self.meta.partition_shape.sig)
        rad_out, rad_in = self.params.ro, self.params.ri
        mask_out = 1*_make_circular_mask(center[1], center[0], sig_shape[1], sig_shape[0], rad_out)
        mask_in = 1*_make_circular_mask(center[1], center[0], sig_shape[1], sig_shape[0], rad_in)
        mask = mask_out - mask_in

        kwargs = {'mask': mask,}
        return kwargs
    
    def process_frame(self, frame):
         self.results.intensity[:] = np.std(frame[self.task_data.mask == 1])         

def run_fem(ctx, dataset, center, ri, ro, roi=None):
    udf = FEMUDF(center=center, ri=ri, ro=ro)
    pass_results = ctx.run_udf(dataset=dataset, udf=udf, roi=roi)
    return pass_results

    
def make_circle(r):
    t = np.arange(0, np.pi * 2.0, 0.01)
    t = t.reshape((len(t), 1))
    x = r * np.cos(t)
    y = r * np.sin(t)
    return np.hstack((x, y))

def draw_circle(ax, radius=(10,10), center=(64,64), facecolor=(1,0,0,1), edgecolor='None'):
    ## https://matplotlib.org/stable/gallery/shapes_and_collections/donut.html#
    
    ri, ro=radius
    inside_vertices = make_circle(ri)
    outside_vertices = make_circle(ro)
    
    cx, cy=center

    Path = mpath.Path
    codes = np.ones(len(inside_vertices), dtype=mpath.Path.code_type) * mpath.Path.LINETO
    codes[0] = mpath.Path.MOVETO
    vertices = np.concatenate((outside_vertices[::1], inside_vertices[::-1]))
    #print(vertices.shape)
    
    vertices[:, 0] += cx # x move
    vertices[:, 1] += cy # y move
    all_codes = np.concatenate((codes, codes))
    path = mpath.Path(vertices, all_codes)
    patch = mpatches.PathPatch(path, facecolor=facecolor, edgecolor=edgecolor)
    ax.add_patch(patch)

In [ ]:
def make_mask(img_angle, step, shift, threshold=100):
    #img_angle = np.angle('img from radial fourier analysis') (range: -np.pi ~ np.pi, -180 deg ~ 180 deg)
    #step = to divide the range 
    #shift = n th of divided steps 
    #example-> (step=9): each 40 degree / (shift=0): from -180 degree to -140 degree
    
    v_min= -np.pi+np.pi*(shift)/step*2    
    v_max=-np.pi+np.pi*(shift+1)/step*2   

    threshold = (v_min<img_angle) & (img_angle<v_max)

    thresholded = closing(opening(threshold)) # from skimage, remove noise or background

    label_img= label(thresholded)
    region=regionprops(label_img)

    mask=[]

    for i in range(1, len(region)+1):
        if np.sum(label_img==i)>threshold:
            mask.append(label_img==i)

    return mask

def list_append(main_list, elem_list):
    for elem in elem_list:
        main_list.append(elem)
    return main_list


## Figure 4. Virtual ring detector method

In [ ]:
start=time.time()

fig = plt.figure(dpi=300, layout='tight')

print('##4D STEM image##')
img_file=r'C:\Users\j.park\Documents\python\Zn deposition\4D_STEM\data\06_1_deposited.xml'
ds=ctx.load("auto", path=img_file)
img_w, img_h, diff_w, diff_h= ds.shape # navigation shape (x, y) , signal shape (x, y)
print('image size: '+str(img_w)+' x '+str(img_h))
print('diff size: '+str(diff_w)+' x '+str(diff_h))

ax=fig.add_subplot(231)
ring = ctx.create_ring_analysis(dataset=ds, ri=70, ro=100)
ring_res=ctx.run(ring)
ring_res.intensity_log.raw_data[0][0]=0
ax.imshow(ring_res.intensity_log.raw_data, cmap='gray')
# res_pixelsum=ctx.run_udf(dataset=ds, udf=SumOfPixels())
# res_pixelsum['sum_of_pixels'].data[0][0]=0
# ax.imshow(res_pixelsum['sum_of_pixels'], cmap='gray')
ax.axis('off')
ax.set_title('(a) Virtual dark field', fontsize=8)
scalebar(ax, scale=(1e-08), img_w=img_w, img_h=img_h, length=500, unit='nm', fontsize=5)

ax0=fig.add_subplot(232)
res_max = ctx.run_udf(dataset=ds, udf=MaxFrameUDF())#, progress=True)
ax0.imshow(np.log1p(res_max['maxframe']), cmap='gray')
ax0.axis('off')
ax0.set_title('(b) Merged diffraction pattern', fontsize=8)
#scalebar(ax0, scale=(0.14), img_w=img_w, img_h=img_h, length=5, unit='1/nm', fontsize=5)

center=(63, 64)
vmin=0
vmax=0.5

color={
        0:'gray',
        1:'Reds',
        2:'Greens',
        3:'Blues'
}
    
radius=[(0,7), (29, 36), (38, 46), (49,56)]
titles=['(c) Center beam', '(d) Zn (100) & (101)', '(e) Zn (102)', '(f) Zn (103) & (110)']

def rgb(n, t=0.3):
    if n==0:
        return (0,0,0,0.5)
    else:
        r=(n%3==1)
        g=(n%3==2)
        b=(n%3==0)
        return (r,g,b,t)

for i in range(len(radius)):
    draw_circle(ax0, radius[i], center=center, facecolor=rgb(i)) #(1,0,0,0.3))
    
    if i>0: 
    #    draw_circle(ax0, radius[i], center=center, facecolor=rgb(i)) #(1,0,0,0.3))
        step=10
    else:
        step=1
    # Inner ring masked virtual HADDF image
    ax=fig.add_subplot(233+i)
    ri, ro=radius[i]
    #ring = ctx.create_ring_analysis(dataset=ds, ri=ri, ro=ro)
    fem=FEMUDF(center=center, ri=ri, ro=ro)
    fem_res=ctx.run_udf(dataset=ds, udf=fem)
    fem_res['intensity'].raw_data[0]=0

    vmin=min(fem_res['intensity'].raw_data[1:].ravel())/step
    vmax=max(fem_res['intensity'].raw_data[1:].ravel())/step
    ax.imshow(fem_res['intensity'], cmap=color[i], vmin=vmin, vmax=vmax) #cmap='gray')
    ax.axis('off')
    ax.set_title(titles[i], fontsize=8)
    
scalebar(ax0, scale=(0.14), img_w=diff_w, img_h=diff_h, length=5, unit='1/nm', fontsize=5)

end=time.time()
print('processing time: ', end-start, '[sec]')

In [ ]:
## Colorbar

#plt.imshow(np.outer(np.ones(10), np.arange(100)), cmap='Reds')
plt.imshow(np.outer(np.ones(10), np.arange(100)), cmap='Greens')
#plt.imshow(np.outer(np.ones(10), np.arange(100)), cmap='Blues')

plt.axis('off')

In [ ]:
#Calibration for 4DSTEM image

fig = plt.figure(figsize=(5, 5), dpi=300, layout='tight')

####################
# Normal STEM image
####################
print('##Normal STEM image##')
img_file=r'C:\Users\j.park\Desktop\TEM data\4D_STEM_Zn_deposition\Zn_new\20230809 0924 STEM 13000 x HAADF.emd'

imgs=nio.read(img_file)
img_w, img_h = imgs['data'].shape
print('Size of image: ' + str(img_w) + ' x ' + str(img_h))
scale=imgs['pixelSize'][0]
print('1 pixel: '+str(scale) + ' ' +imgs['pixelUnit'][0])

ax=fig.add_subplot(111)
ax.imshow(imgs['data'], cmap='gray')
ax.axis('off')
scalebar(ax, scale=(scale*(10**(-9))), img_w=img_w, img_h=img_h, length=1, fontsize=10)

In [ ]:
i=1

fig = plt.figure(dpi=300, layout='tight')
ri, ro=radius[i]
#ri, ro=70,100
fem=FEMUDF(center=center, ri=ri, ro=ro)
fem_res=ctx.run_udf(dataset=ds, udf=fem)
fem_res['intensity'].raw_data[0]=0

step=10
vmin=min(fem_res['intensity'].raw_data[1:].ravel())/step
vmax=max(fem_res['intensity'].raw_data[1:].ravel())/step

ax1=fig.add_subplot(121)
ax1.imshow(fem_res['intensity'], cmap='gray', vmin=vmin, vmax=vmax)

ax2=fig.add_subplot(122)
ax2.hist(fem_res['intensity'].raw_data.ravel(), bins=255, range=(vmin,vmax))

print(min(fem_res['intensity'].raw_data[1:].ravel()))
print(max(fem_res['intensity'].raw_data[1:].ravel()))

## Figure 5. Radial Fourier analysis method

In [ ]:
start=time.time()

fig = plt.figure(dpi=300, layout='tight')

## Load 4D STEM data and plot virtual dark field image
print('##4D STEM image##')
img_file=r'C:\Users\j.park\Documents\python\Zn deposition\4D_STEM\data\06_1_deposited.xml'
ds=ctx.load("auto", path=img_file)
img_w, img_h, diff_w, diff_h= ds.shape # navigation shape (x, y) , signal shape (x, y)
print('image size: '+str(img_w)+' x '+str(img_h))
print('diff size: '+str(diff_w)+' x '+str(diff_h))

## Radial Fourier analysis to construct angle assigned image (2D diffraction pattern => angle value: -180 ~ +180 degree)
rfa = ctx.create_radial_fourier_analysis(
    dataset=ds,
    cx=63,
    cy=64,
    ri=0,
    ro=64,
    n_bins=64,   # y_range: Radius [px] (ro-ri)
    max_order=10 # x_range: Order of fourier component (depending on computing power)
)
res = ctx.run(rfa)

ri, ro = 29, 56 #30, 60 #Band-pass filter
maps = res.raw_results[ri:ro].sum(axis=0)
img_angle = np.angle(maps[2]) #even numbers work

## Plot angle values in image
ax0=fig.add_subplot(231)
ax0.imshow(img_angle, cmap='jet')
ax0.axis('off')
ax0.set_title('(a) Angle map of radial Fourier analysis', fontsize=8)


ax0=fig.add_subplot(232)
blank_img=np.ones((img_w, img_h))
ax0.imshow(blank_img, cmap='gray', vmin=0, vmax=1)
ax0.axis('off')
ax0.set_title('(b) Thresholding by 9 angle ranges', fontsize=8)

ax=fig.add_subplot(233)
ring = ctx.create_ring_analysis(dataset=ds, ri=70, ro=100)
ring_res=ctx.run(ring)
ring_res.intensity_log.raw_data[0][0]=0
ax.imshow(ring_res.intensity_log.raw_data, cmap='gray')
ax.axis('off')
ax.set_title('(c) Virtual dark field with domains', fontsize=8)
scalebar(ax, scale=(1e-08), img_w=img_w, img_h=img_h, length=500, unit='nm', fontsize=5)

## Find top 9 mask(domain) candidates, larger than minimum mask area
step=9
max_mask=9
min_area=100
thres_mask=[]
area_mask=np.zeros((max_mask), dtype=int)

def min_check(target_list, num):
    for i in range(len(target_list)):
        if target_list[i]>num:
            return (i-1)
    return i

for shift in range(step):
    ## Thresholding angle by the current range
    v_min= -np.pi+np.pi*(shift)/step*2    
    v_max=-np.pi+np.pi*(shift+1)/step*2
    threshold = (v_min<img_angle) & (img_angle<v_max)
    thresholded = closing(opening(threshold)) # from skimage, remove noise or background
    label_img= label(thresholded)
    region=regionprops(label_img)
    
    ## Find top 9 domain areas
    for i in range(1, len(region)+1):
        temp_area=np.sum(label_img==i)
        if temp_area>min_area:
            pos=min_check(area_mask, temp_area)
            if pos>-1:
                if len(thres_mask)<max_mask:
                    for j in range(pos):
                        area_mask[j]=area_mask[j+1]
                    area_mask[pos]=temp_area
                    thres_mask.insert(max_mask-len(thres_mask)-pos-1, label_img==i)
                else:
                    for j in range(pos):
                        area_mask[j]=area_mask[j+1]
                        thres_mask[j]=thres_mask[j+1]
                    area_mask[pos]=temp_area
                    thres_mask[pos]=(label_img==i)

## Find top 3 mask(domain) with best diffraction patterns
max_figures=3
std_diff=np.zeros((max_figures), dtype=float)
masked_diff=np.zeros((max_figures, img_w, img_h))
top_mask=np.zeros((max_figures), dtype=int)

for i in range(len(thres_mask)):
    res_max_mask = ctx.run_udf(dataset=ds, udf=MaxFrameUDF(), roi=thres_mask[len(thres_mask)-i-1])
    temp_std=np.std(np.log1p(res_max_mask['maxframe']))
    pos=min_check(std_diff, temp_std)
    if pos>-1:
        for j in range(pos):
            std_diff[j]=std_diff[j+1]
            top_mask[j]=top_mask[j+1]
            masked_diff[j]=masked_diff[j+1]
        std_diff[pos]=temp_std
        top_mask[pos]=i
        masked_diff[pos]=np.log1p(res_max_mask['maxframe']) 

color_list=['Reds', 'Greens', 'Blues', 'Oranges', 'Purples']

## plot diffraction patterns as figure
if max_figures>len(masked_diff):
    max_figures=len(masked_diff)

for j in range(max_figures):
    ax1=fig.add_subplot(234+j)
    ax1.imshow(masked_diff[max_figures-j-1], cmap='gray')
    ax1.axis('off')
    ax1.set_title('('+chr(100+j)+') DP from '+color_list[j][:-1] + ' domain', fontsize=8)
    scalebar(ax1, scale=(0.14), img_w=diff_w, img_h=diff_h, length=5, unit='1/nm', fontsize=5, text=False)

    
## plot mask(domain) location
for i in range(max_figures):
    mask=np.ma.masked_where(thres_mask[len(thres_mask)-top_mask[max_figures-i-1]-1]==0, thres_mask[len(thres_mask)-top_mask[max_figures-i-1]-1])
    ax.imshow(mask, color_list[i], interpolation='none', alpha=0.8, vmin=0, vmax=1)

end=time.time()
print('processing time: ', end-start, '[sec]')

In [ ]:
#img_angle = np.angle(maps[2])
fig = plt.figure(figsize=(5,5) ,dpi=300, layout='tight')
step=9 #default=9
shift=0 #recommend: next 9 images if step>9
for i in range(step):
	v_min= -np.pi +np.pi*(i+shift)/step*2    
	v_max= -np.pi +np.pi*(i+shift+1)/step*2 

	img_thres=(v_min<img_angle) & (img_angle<v_max)

	ax=fig.add_subplot(330 + 1 + i) #subplot(xy0) => x*y = group<10
	ax.imshow(img_thres, cmap='gray')
	ax.axis('off')
	#plt.imshow(img_angle, cmap='gray', vmin=v_min, vmax=v_max)

In [ ]:
## Colorbar

import matplotlib.pyplot as plt
import numpy as np

# Generate dataset for the plot
x = np.linspace(-10, 10, 1000)
y = np.sin(x)

# Use the rdbu colormap to color the background
plt.imshow(np.outer(np.ones(10), np.arange(100)), cmap='jet')

plt.axis('off')
# Show the plot
plt.show()

## SI

### Orientation mapping: Step by step

In [ ]:
print('##4D STEM image##')
img_file=r'C:\Users\j.park\Documents\python\Zn deposition\4D_STEM\data\06_1_deposited.xml'
ds=ctx.load("auto", path=img_file)
img_w, img_h, diff_w, diff_h= ds.shape # navigation shape (x, y) , signal shape (x, y)
print('image size: '+str(img_w)+' x '+str(img_h))
print('diff size: '+str(diff_w)+' x '+str(diff_h))


# Analysis: Radial fourier analysis (diffraction pattern converts to angle value)
rfa = ctx.create_radial_fourier_analysis(
    dataset=ds,
    cx=63,
    cy=64,
    ri=0,
    ro=64,
    n_bins=64,   # y_range: Radius [px] (ro-ri)
    max_order=10 # x_range: Order of fourier component (depending on computing power)
)
res = ctx.run(rfa)#, progress=True)
#res.keys  #dominant_0, absolute_y_x, phase_y_x, complex_y_x

# Band-pass filter
ri, ro = 29, 56 #30, 60 #Band-pass filter
maps = res.raw_results[ri:ro].sum(axis=0)

# Convert to image with angle values
img_angle = np.angle(maps[2])

In [ ]:
plt.imshow(img_angle, cmap='jet')
plt.colorbar() # range: -pi ~ +pi [rad] (same as 0~360 degree)

In [ ]:
#view few images 
ri, ro = 29, 56 #Band-pass filter
#30, 60

maps = res.raw_results[ri:ro].sum(axis=0)

absmaps=np.abs(maps[1:]/np.abs(maps[0]))
vmin=absmaps[1:].min()
vmax=absmaps[1:].max()

figure=plt.figure(figsize=(10,10))

for i in range(9):
    figure.add_subplot(330+i+1)
    plt.imshow(absmaps[i], vmin=vmin, vmax=vmax)
    plt.axis('off')

In [ ]:
#Screening the proper angle range to seperate the grains

#img_angle = np.angle(maps[2])
fig = plt.figure(figsize=(6,6), dpi=300, layout='tight')

step=9 #default=9
shift=0 #recommend: next 9 images if step>9
for i in range(step):
	v_min= -np.pi +np.pi*(i+shift)/step*2    
	v_max= -np.pi +np.pi*(i+shift+1)/step*2 

	img_thres=(v_min<img_angle) & (img_angle<v_max)

	ax=fig.add_subplot(330 + 1 + i) #subplot(xy0) => x*y = group<10
	ax.imshow(img_thres, cmap='gray')
	ax.axis('off')
	#plt.imshow(img_angle, cmap='gray', vmin=v_min, vmax=v_max)

plt.suptitle('(a) Thresholded by 40 degrees (360/9)', fontsize=15)
#plt.show()

In [ ]:
step=9
max_mask=9
min_area=100
thres_mask=[]
area_mask=np.zeros((max_mask), dtype=int)

def min_check(target_list, num):
    for i in range(len(target_list)):
        if target_list[i]>num:
            return (i-1)
    return i

for shift in range(step):
    ## Thresholding angle by the current range
    v_min= -np.pi+np.pi*(shift)/step*2    
    v_max=-np.pi+np.pi*(shift+1)/step*2
    threshold = (v_min<img_angle) & (img_angle<v_max)
    thresholded = closing(opening(threshold)) # from skimage, remove noise or background
    label_img= label(thresholded)
    region=regionprops(label_img)
    
    ## Find top 9 domain areas
    for i in range(1, len(region)+1):
        temp_area=np.sum(label_img==i)
        if temp_area>min_area:
            pos=min_check(area_mask, temp_area)
            if pos>-1:
                if len(thres_mask)<max_mask:
                    for j in range(pos):
                        area_mask[j]=area_mask[j+1]
                    area_mask[pos]=temp_area
                    thres_mask.insert(max_mask-len(thres_mask)-pos-1, label_img==i)
                else:
                    for j in range(pos):
                        area_mask[j]=area_mask[j+1]
                        thres_mask[j]=thres_mask[j+1]
                    area_mask[pos]=temp_area
                    thres_mask[pos]=(label_img==i)

fig = plt.figure(figsize=(6,6), dpi=300, layout='tight')    
for i in range(len(thres_mask)):
    ax=fig.add_subplot(330 + 1 + i)
    #plt.subplot(220 + 1 + i)
    ax.imshow(thres_mask[len(thres_mask)-i-1], cmap='gray')
    ax.axis('off')

plt.suptitle('(b) Segmented domain candidates (sorted by area)', fontsize=15)

In [ ]:
color_list=['Reds', 'Greens', 'Blues', 'Oranges', 'Purples']
max_figures=5
std_diff=np.zeros((max_figures), dtype=float)
masked_diff=np.zeros((max_figures,img_w,img_h))
top_mask=np.zeros((max_figures), dtype=int)

for i in range(len(thres_mask)):
    res_max_mask = ctx.run_udf(dataset=ds, udf=MaxFrameUDF(), roi=thres_mask[len(thres_mask)-i-1])#, progress=True)
    temp_std=np.std(np.log1p(res_max_mask['maxframe']))
    pos=min_check(std_diff, temp_std)
    if pos>-1:
        for j in range(pos):
            std_diff[j]=std_diff[j+1]
            top_mask[j]=top_mask[j+1]
            masked_diff[j]=masked_diff[j+1]
        std_diff[pos]=temp_std
        top_mask[pos]=i
        masked_diff[pos]=np.log1p(res_max_mask['maxframe'])

fig, axs = plt.subplots(2, 5, figsize=(10, 5), dpi=300, layout='tight')                      

for j in range(max_figures):
    axs[0, j].imshow(masked_diff[max_figures-j-1], cmap='gray')
    axs[0, j].axis('off')
    axs[0, j].set_title('std: {0:.3}'.format(std_diff[max_figures-j-1]), fontsize=12)
#    '{0:.3g}'.format(num) 
    
    axs[1, j].imshow(thres_mask[len(thres_mask)-top_mask[max_figures-j-1]-1], cmap='gray')
    axs[1, j].axis('off')
    axs[1, j].set_title(color_list[j][:-1], fontsize=12)
    
fig.suptitle('(c) Selected top 5 DPs & Domains', fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)

img=np.abs(res.raw_results).sum(axis=(0,1))
img[0][0]=0
ax.imshow(img, 'gray', interpolation='none')

color_list=['Reds', 'Greens', 'Blues', 'Oranges', 'Purples']

for i in range(5):
    mask=np.ma.masked_where(thres_mask[len(thres_mask)-top_five_mask[max_figures-i-1]-1]==0, thres_mask[len(thres_mask)-top_five_mask[max_figures-i-1]-1])
    ax.imshow(mask, color_list[i],interpolation='none', alpha=0.8, vmin=0, vmax=1)

ax.axis('off')

fig.suptitle('(d) Top 5 domains mapped image', fontsize=15)

In [ ]:
plt.imshow(masked_diff[4], cmap='gray')
plt.axis('off')

### Calibration with Au nanoparticle

In [ ]:
img_file='data/Au_NP/20230808_5_standard_sample.xml'

ds=ctx.load("auto", path=img_file)

img_w, img_h, diff_w, diff_h= ds.shape # navigation shape (x, y) , signal shape (x, y)
print('image size: '+str(img_w)+' x '+str(img_h))
print('diff size: '+str(diff_w)+' x '+str(diff_h))

fig = plt.figure(dpi=300)
ax0=fig.add_subplot(121)
res_max = ctx.run_udf(dataset=ds, udf=MaxFrameUDF()) #, progress=True)
ax0.imshow(np.log1p(res_max['maxframe']), cmap='gray')
ax0.axis('off')
ax0.set_title('Merged diffraction pattern', fontsize=8)

radius=[(15.5, 20.5)]
center=(64, 65.5)
draw_circle(ax0, radius[0], center=center, facecolor=(1,0,0,0.3))

scalebar(ax0, scale=(0.236), img_w=img_w, img_h=img_h, length=5, unit='1/nm', fontsize=10)

ax=fig.add_subplot(122)
ri, ro=radius[0]
fem=FEMUDF(center=center, ri=ri, ro=ro)
fem_res=ctx.run_udf(dataset=ds, udf=fem)
ax.imshow(fem_res['intensity'], cmap='gray')
ax.axis('off')